In [1]:
import itertools
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

from handcrafted.app.dataset.dataset_loader import DatasetLoader
from handcrafted.app.dataset.utils.dataset_creator import DatasetCreator
from handcrafted.app.dataset.utils.frames_splitter import FramesSplitter
from handcrafted.app.model.model_statistics import ModelStatistics

2025-04-01 15:01:51.238111: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-01 15:01:51.239602: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-01 15:01:51.260250: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-01 15:01:51.260288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-01 15:01:51.261155: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
loader = DatasetLoader(directory="./data/frames_no_bg/")

12023it [00:11, 1021.67it/s]


In [3]:
num_signers = 8
if num_signers == -1:
    signers = loader.signers
    num_signers = loader.num_signers
else:
    signers = dict(itertools.islice(loader.signers.items(), num_signers))
splitter = FramesSplitter(signers, val_split=0.3, test_split=0.3, frames_to_extract=1000, seed=42)

In [4]:
for signer in signers.values():
    signer_frames = []
    print(f"Signer {signer.id} has {len(signer.videos)} videos")

Signer 117 has 28 videos
Signer 26 has 93 videos
Signer 63 has 39 videos
Signer 35 has 55 videos
Signer 42 has 34 videos
Signer 48 has 6 videos
Signer 104 has 4 videos
Signer 70 has 14 videos


In [5]:
X_train, X_train_aug, y_train, X_val, X_val_aug, y_val, X_test, X_test_aug, y_test = splitter.split(X_content=lambda f: f)

100%|██████████| 8/8 [00:00<00:00, 542.74it/s]


In [6]:
# Get the total of different labels
num_classes = num_signers
print(num_classes)

8


In [7]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.fit_transform(y_val)
y_test_encoded = label_encoder.fit_transform(y_test)

In [8]:
BATCH_SIZE = 4096
dataset_creator = DatasetCreator()
train_dataset = dataset_creator.create_custom_dataset(X_train, y_train_encoded, augmentation = X_train_aug, batch_size=BATCH_SIZE)
val_dataset = dataset_creator.create_custom_dataset(X_val, y_val_encoded,  augmentation=X_val_aug, batch_size=BATCH_SIZE)
test_dataset = dataset_creator.create_custom_dataset(X_test, y_test_encoded, augmentation=X_test_aug, batch_size=BATCH_SIZE)


100%|██████████| 2/2 [00:00<00:00, 104857.60it/s]


In [9]:
xgb_params = {
    "objective":"multi:softmax",
    "num_class":num_classes,
    "eval_metric":"mlogloss",
    "max_depth":6,
    "learning_rate":0.1,
    "n_estimators":100,
    "subsample":0.8,
    "colsample_bytree":0.8,
    "gamma":0,
    "min_child_weight":1,
    "reg_lambda":1,
    "reg_alpha":0,
    "tree_method":"hist", # Change to "gpu_hist" for GPU acceleration
    # "device":"cuda", # Use GPU
}

In [10]:
xgb_model = None
for train_batch in tqdm(val_dataset):
    X_batch, y_batch = train_batch.load(shuffle=True)
    dtrain = xgb.DMatrix(X_batch, label=y_batch)
    xgb_model = xgb.train(params=xgb_params, dtrain=dtrain, num_boost_round=10, xgb_model=xgb_model)

  0%|          | 0/2 [05:23<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
y_pred = []
y_test_full = []
for test_batch in tqdm(test_dataset):
    X_batch, y_batch = test_batch.load(shuffle=True)
    y_test_full.extend(y_batch)
    dtest = xgb.DMatrix(X_batch)
    y_pred_batch = xgb_model.predict(dtest)
    y_pred.extend(y_pred_batch)

In [ ]:
stats = ModelStatistics(save_name=f"svc_signer_test_{len(y_test_full)}", save_dir="signer/plots")
stats.print_accuracy(y_test_full, y_pred)
stats.plot_confusion_matrix(y_test_full, y_pred, save=True, plot=True)